In [1]:
import os
import faiss
import numpy as np
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings("ignore")



c:\Users\lovel\miniconda3\envs\med-chatbot\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
# 1. Load and process the medical book (PDF)
def load_pdf(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [4]:
# 2. Split the text into chunks
def split_text_into_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

In [5]:
# 3. Download the pre-trained HuggingFace embedding model
def download_embeddings_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [6]:
# 4. Create FAISS index from the embeddings
def create_faiss_index(text_chunks, embeddings):
    # Convert chunks into embeddings
    chunk_texts = [chunk.page_content for chunk in text_chunks]
    chunk_embeddings = embeddings.embed_documents(chunk_texts)

    # Initialize FAISS index
    dimension = 384  # Embedding dimension from MiniLM-L6-v2
    index = faiss.IndexFlatL2(dimension)

    # Convert embeddings to a numpy array and add to the FAISS index
    embedding_array = np.array(chunk_embeddings).astype('float32')
    index.add(embedding_array)

    # Return the index and text chunks for retrieval
    return index, chunk_texts


In [7]:
# 5. Search the FAISS index for the most similar text chunk based on query
def search_index(query, index, embeddings, chunk_texts):
    # Embed the query
    query_embedding = embeddings.embed_query(query)

    # Search for the most similar text in the index
    D, I = index.search(np.array([query_embedding], dtype='float32'), k=1)  # Find the top match

    # Get the matching chunk
    matching_chunk = chunk_texts[I[0][0]] if I[0].size > 0 else "No relevant information found."
    
    return matching_chunk

In [8]:
# 6. Display the response clearly and appropriately
def display_response(matching_chunk):
    # Ensure that the response is clear and concise
    max_length = 1000  # Define a maximum length for response
    if len(matching_chunk) > max_length:
        # Truncate long responses
        response = matching_chunk[:max_length] + "..."
    else:
        response = matching_chunk
    
    # Print the response
    print("Here is the relevant response to your query:\n")
    print(response)
    print("\nIf you need more details, feel free to ask another question!")

In [11]:
# Main function to run the chatbot
def run_chatbot():
    data_path = "data/"
    
    # 1. Load the PDF
    documents = load_pdf(data_path)

    # 2. Split the text into chunks
    text_chunks = split_text_into_chunks(documents)

    # 3. Download the HuggingFace embedding model
    embeddings = download_embeddings_model()

    # 4. Create FAISS index
    index, chunk_texts = create_faiss_index(text_chunks, embeddings)

    # 5. Simulate a chat session
    while True:
        query = input("Ask a medical question (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            print("Exiting the chatbot. Have a great day!")
            break
        
        matching_chunk = search_index(query, index, embeddings, chunk_texts)
        
        # Display the result in a clear and concise manner
        display_response(matching_chunk)
    
# Run the chatbot
run_chatbot()

c:\Users\lovel\miniconda3\envs\med-chatbot\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\lovel\miniconda3\envs\med-chatbot\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lovel\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mod

Here is the relevant response to your query:

consistently identifiable cause, but researchers havesuggested that migraines or transient ischemic attacksmay be the trigger. (A transient ischemic attack ,

If you need more details, feel free to ask another question!
Exiting the chatbot. Have a great day!
